In [18]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer
from generation_dataset import Tweet_SympDataset
import textwrap
import argparse
#from prompt_design import design0
import pandas as pd
from os import listdir

In [19]:
check_point_dir = 'generation_checkpoint1/'
PATH =  check_point_dir + listdir(check_point_dir)[0]
generation_result = 'generated_text/generation_result1.pickle'


In [20]:
USE_GPU = True
args_dict = dict(
    data_dir='',  # データセットのディレクトリ
    model_name_or_path="sonoisa/t5-base-japanese",
    tokenizer_name_or_path="sonoisa/t5-base-japanese",

    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    gradient_accumulation_steps=1,

    max_input_length=512,
    max_target_length=512,
    train_batch_size=8,
    eval_batch_size=8,
    num_train_epochs=24,
    n_gpu=1 if USE_GPU else 0,
    early_stop_callback=False,
    fp_16=False,
    opt_level='O1',
    max_grad_norm=1.0
)

args = argparse.Namespace(**args_dict)

In [21]:
PRETRAINED_MODEL_DIR = "sonoisa/t5-base-japanese"
from generation_experiment import T5FineTuner
model_tuner = T5FineTuner.load_from_checkpoint(checkpoint_path=PATH,name_space =args)
trained_model = model_tuner.model
tokenizer = model_tuner.tokenizer

In [22]:
print(trained_model.device)

cpu


In [23]:
MAX_SOURCE_LENGTH = 50  # 入力される記事本文の最大トークン数
MAX_TARGET_LENGTH = 150  # 生成されるタイトルの最大トークン数

In [24]:
# https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja から引用・一部改変
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]+', '〜', s)  # normalize tildes (modified by Isao Sonobe)
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s
    
import math
def normalize_text(text):
    assert "\n" not in text and "\r" not in text
    text = text.replace("\t", " ")
    text = text.strip()
    text = normalize_neologd(text)
    text = text.lower()
    return text

def preprocess_answer(text):
    return normalize_text(text.replace("\n", ""))


In [37]:
def generate_bodies(num,prompt):
    inputs = [preprocess_answer(prompt)]
    batch = tokenizer.batch_encode_plus(
    inputs, max_length=MAX_SOURCE_LENGTH, truncation=True, 
    padding="longest", return_tensors="pt")
    input_ids = batch['input_ids']
    input_mask = batch['attention_mask']
    outputs = trained_model.generate(
        input_ids=input_ids, attention_mask=input_mask, 
        max_length=MAX_TARGET_LENGTH,
        temperature=1.0,  # 生成にランダム性を入れる温度パラメータ
        #num_beams= num,  # ビームサーチの探索幅
        #diversity_penalty=0.5,  # 生成結果の多様性を生み出すためのペナルティパラメータ
        #This value is subtracted from a beam’s score if it generates a token same as any beam from other group at a particular time. Note that diversity_penalty is only effective if group beam search is enabled.
        #num_beam_groups= num,  # ビームサーチのグループ
        do_sample=True, 
        #top_k=30,
        num_return_sequences= num,  # 生成する文の数
        repetition_penalty=8.0,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
    )

    # 生成されたトークン列を文字列に変換する
    generated_texts = [tokenizer.decode(ids, skip_special_tokens=True, 
                                     clean_up_tokenization_spaces=False) 
                    for ids in outputs]
    return generated_texts

In [39]:
prompt = 'ステロイド使用のTweetは？<extra_id_0>'
generate_bodies(100,prompt)

["'<user_name> ありがとうございます\\nステロイドの副作用で食欲が増して、お腹も空いてきました'",
 "'<user_name> 私もステロイドの副作用で不眠症になりました\\u200d'",
 "'<user_name> 私もアトピー性皮膚炎です\\nステロイド塗るとすぐ治りますよ!'",
 "'<user_name> ありがとうございます\\u200d♂\\nステロイドの副作用で骨粗症になりやすいって言われたんですけど、そんなに酷くはないみたいです!'",
 "'<user_name> ありがとうございます\\nステロイドの点滴で少し落ち着きましたが、まだ血糖値が下がらないです...'",
 "'<user_name> 私も麻疹が酷くて、皮膚科でステロイド剤を処方してもらいました。\\n早く治りますように!'",
 "'<user_name> 私もアトピー持ちでした。\\nステロイド使ってたので、皮膚科に行けば良かったです'",
 "'<user_name> ありがとうございます\\u200d♂\\nステロイドの副作用で骨粗症になりやすいって言われてたんですけど、先生に相談してみようと思います。'",
 "'<user_name> 私も喘息のステロイド吸入薬使ってます\\nめちゃくちゃ痛いですよね...'",
 "'<user_name> ありがとうございます\\u200d♂\\nステロイドが効いてるみたいで、だいぶ楽になりました!'",
 "'<user_name> ステロイドは免疫力を低下させる作用もあるので、風邪やインフルエンザにかかっても抗ウイルス薬を処方してくれます。\\nコロナが流行り始めて2ヶ月ほど経ちましたが、まだ咳はまだ残っています'",
 "'<user_name> おはよーございます\\u200d♂\\nステロイドの副作用で顔がパンパンに浮腫んでるんですけど、そのせいかめちゃくちゃ眠いです(&gt;_&lt;)'",
 "'<user_name> 私もステロイドの副作用で不眠症になりました\\n今は薬が効いてる感じです(^-^)/'",
 "'ステロイドを塗ると皮膚が黒くなるの?\\n皮膚が色素沈着を起こすのは炎症で皮膚が赤黒くなるから、そうなる前に短期で皮膚を治療すれば色素沈着は起こりにくいんだって。\\nつまりステロイ

In [ ]:
'<user_name> お疲れ様です\\nステロイドの副作用で食欲増進があるみたいですね。'",
 "'<user_name> ありがとうございます\\u200d♂\\nステロイドの副作用で骨粗症になりやすいみたいです。'",
 "'<user_name> ありがとうございます\\u200d♂\\nステロイドが効いてるみたいで、少し楽になってきました!'",
 "'<user_name> おはよーございます\\u200d\\nステロイドの副作用で食欲亢進と満腹感が凄くて、食べる量も減ってるんですけどね(&gt;_&lt;)'",
 "'<user_name> お大事になさってくださいね。\\n私もアトピー性皮膚炎なので、ステロイドの塗り薬を処方してもらいました。'",
 "'<user_name> ステロイドの副作用で骨粗症になりました。'",
 "'<user_name> ありがとうございます\\u200d♂\\nステロイドの副作用で骨粗症になってしまって、薬を飲んでるんですが、今朝はお腹に力が入らなくて倒れてました'"

In [29]:
len(tokenizer.encode("'アトピー性皮膚炎の方はご注意を!\\u3000グリチルリチン酸ジカリウム(2)配合薬用化粧水で脱ステロイドすると強い副作用/副反応のリバウンドが! <url>'"))

47

In [ ]:

# 推論モード設定
trained_model.eval()

all_bodies =[]
all_index = []

# 生成処理を行う
for s in ss:
    index = s[0]
    num = s[1][0]
    symp_text = s[1][1]
    additional = round(num*0.35)
    num += additional
    bodies = generate_bodies(num,symp_text)
    for body in bodies:
        all_index.append(index)
        all_bodies.append(body)

In [ ]:
#生成された文とidのdf
body_index_df2 = pd.DataFrame({'generated_tweet':all_bodies,'id':all_index})

In [ ]:
train_file = r'../MedWeb_TestCollection/NTCIR-13_MedWeb_ja_training.xlsx'
test_file = r'../MedWeb_TestCollection/NTCIR-13_MedWeb_ja_test.xlsx'
org_train = pd.read_excel(train_file,sheet_name =  'ja_train')
org_test = pd.read_excel(test_file,sheet_name="ja_test")
original_df = pd.concat([org_train, org_test], axis=0,ignore_index=True)

In [ ]:
original_tweet_set = set(list(original_df['Tweet']))

In [ ]:
def replace_original(x):
    if x in original_tweet_set:
        return 'inoriginal'
    else:
        return x

In [ ]:
body_index_df['generated_tweet'] = body_index_df['generated_tweet'].map(lambda x:replace_original(x))

In [ ]:
subtracted_df = body_index_df[body_index_df['generated_tweet'] != 'inoriginal']
in_original_df = body_index_df[body_index_df['generated_tweet'] == 'inoriginal']

In [ ]:
subtracted_df = subtracted_df.sample(n=len(org_train))

In [ ]:
in_original_df[300:]

In [ ]:
def id2label(id):
    return list(bit_pattrns[id])

In [ ]:
subtracted_df['id'] = subtracted_df['id'].map(lambda x:id2label(x))

In [ ]:
final_df = pd.DataFrame({'Tweet':subtracted_df['generated_tweet'], 'labels':subtracted_df['id']})

In [ ]:
print(generation_result)

In [ ]:
import pickle
with open(generation_result,'wb') as f:
    pickle.dump(final_df,f)

以下、例の生成

In [ ]:
import pickle
generation_result = 'generated_text/generation_result1.pickle'
file = generation_result
with open(file, 'rb') as f:
    train_df = pickle.load(f)

In [ ]:
list(train_df['Tweet'])

In [ ]:
train_df.head()

In [ ]:
train_df['labels'] = train_df['labels'].map(lambda x:str(x))

In [ ]:
p = str(list(bit_pattrns[0]))

In [ ]:
train_df[train_df['labels'] == p]

In [ ]:
gen_texts = []
bit_pattrns = [[0,1,0,0,0,0,0,0],[0,0,0,0,0,1,0,0],[0,0,0,0,0,0,1,0]]
for b in bit_pattrns:
    p = str(list(b))
    d = train_df[train_df['labels'] == p]
    if len(d) > 10:
        texts = d.sample(n=10)
        gen_texts.append(texts)

In [ ]:
print('\n'.join(gen_texts[2]['Tweet']))

In [ ]:
import pickle
file = r'../generation_model/generated_text/generation_result2.pickle'
with open(file, 'rb') as f:
    train_df = pickle.load(f)

In [2]:
import pickle 
with open('generated_text2/インスリンgenerated_text2.pickle','rb') as f:
    texts = pickle.load(f)